In [ ]:
import numpy as np                  # Version: 1.24.3
import pandas as pd                 # Version: 1.5.3
import torch                        # Version: 2.5.1
import torch.nn as nn
from   torch.utils.data import DataLoader, Dataset
import sklearn                      # Version: 1.3.0
from   sklearn.metrics import accuracy_score, precision_score, recall_score, matthews_corrcoef, roc_auc_score
from   sklearn.metrics import precision_recall_curve

# Python version: 3.11.4
import os
import random

In [4]:
notebook_dir = os.getcwd()
parent_dir   = os.path.dirname(notebook_dir)
target_dir   = os.path.join(parent_dir, "Data")

def log_and_print(msg, file_handle):
    print(msg)
    file_handle.write(msg + "\n")

### **Training Models A (S1)**

In [ ]:
# MODEL A: Wildfire Prediction using LSTM-AE and Bi-LSTM-AE on Set 1 (S1)

random.seed(36)
np.random.seed(36)
torch.manual_seed(36)
torch.cuda.manual_seed_all(36)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False
g = torch.Generator().manual_seed(36)

SEQUENCE_LENGTHS = {
    "10_day": 10,
    "2_week": 14,
    "monthly": 30
}

NUMERIC_FEATURES = [
    "NDVI_residual", "NDVI_std", "NDVI_trend", "Solar_de-seasonalized",
    "Temperature_residual", "Humidity_trend", "NDVI_var", "Humidity_residual",
    "NDVI_mean", "Temperature_trend", "Temperature [°C]_max",
    "Solar radiation [Jm2/day]_std", "Relative humidity [%]_mean",
    "Temperature [°C]_mean", "Solar radiation [Jm2/day]_sum"
]

CAT_FEATURE = "State"
TARGET_COL  = "fire"

class WildfireDataset(Dataset):
    def __init__(self, df, sequence_length, only_no_fire=True):
        self.sequence_length = sequence_length
        self.data = df.copy()
        self.features = NUMERIC_FEATURES
        self.cat_feature = CAT_FEATURE
        self.target = df[TARGET_COL].values
        grouped = [group for _, group in df.groupby('State', observed=False)]
        self.sequences = []
        self.targets = []
        for g in grouped:
            for i in range(len(g) - sequence_length):
                num_seq = g[self.features].iloc[i:i + sequence_length].values.astype(np.float32)
                cat_seq = g[self.cat_feature].iloc[i:i + sequence_length]
                cat_seq_encoded = cat_seq.astype('category').cat.codes.values.astype(np.float32).reshape(-1, 1)
                full_seq = np.hstack([num_seq, cat_seq_encoded])
                label = g[TARGET_COL].iloc[i + sequence_length - 1] # It predicts the last day of the sequence as fire/no-fire
                window_labels = g[TARGET_COL].iloc[i : i + sequence_length].values
                if only_no_fire and np.any(window_labels != 0):
                    # skip *any* sequence that contains a fire anywhere
                    continue
                self.sequences.append(full_seq)
                self.targets.append(label)
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.float32), torch.tensor(self.targets[idx], dtype=torch.float32)

class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, layer_sizes, num_states, bidirectional=False, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(num_states, embedding_dim)
        self.embedding_dropout = nn.Dropout(dropout)
        # Encoder
        enc_input_dim = input_dim + embedding_dim
        self.encoder_layers = nn.ModuleList()
        self.encoder_dropouts = nn.ModuleList()
        for h in layer_sizes:
            self.encoder_layers.append(
                nn.LSTM(enc_input_dim, h, batch_first=True, bidirectional=bidirectional)
            )
            self.encoder_dropouts.append(nn.Dropout(dropout))
            enc_input_dim = h * (2 if bidirectional else 1)
        # Decoder
        dec_input_dim = enc_input_dim
        self.decoder_layers = nn.ModuleList()
        self.decoder_dropouts = nn.ModuleList()
        for h in reversed(layer_sizes[:-1]):
            self.decoder_layers.append(
                nn.LSTM(dec_input_dim, h, batch_first=True)
            )
            self.decoder_dropouts.append(nn.Dropout(dropout))
            dec_input_dim = h
        # Final projection
        self.output_layer = nn.Linear(dec_input_dim, input_dim + embedding_dim)
        self.tanh = nn.Tanh()
    def forward(self, x):
        # Embedding
        state_idx = x[..., -1].long()
        emb = self.embedding(state_idx)
        emb = self.embedding_dropout(emb)
        seq = torch.cat([x[..., :-1], emb], dim=-1)
        # Encoder
        out = seq
        for lstm, dropout in zip(self.encoder_layers, self.encoder_dropouts):
            out, _ = lstm(out)
            out = dropout(self.tanh(out))
        # Decoder
        for lstm, dropout in zip(self.decoder_layers, self.decoder_dropouts):
            out, _ = lstm(out)
            out = dropout(self.tanh(out))
        # Output
        decoded = self.output_layer(out)
        decoded = self.tanh(decoded)
        return decoded

def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for x, _ in dataloader:
        optimizer.zero_grad()
        x_hat = model(x)
        # Trim output and target to match dimensions
        x_hat_trimmed = x_hat[:, :, :-embedding_dim]
        x_trimmed = x[:, :, :-1]
        loss = criterion(x_hat_trimmed, x_trimmed)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_scores = []
    with torch.no_grad():
        for x, y in dataloader:
            x_hat = model(x)
            # Trim output and target to match dimensions
            x_hat_trimmed = x_hat[:, :, :-embedding_dim]
            x_trimmed = x[:, :, :-1]
            re = ((x_hat_trimmed - x_trimmed) ** 2).mean(dim=(1, 2))
            all_scores.extend(re.numpy())
            all_labels.extend(y.numpy())
    return np.array(all_scores), np.array(all_labels)

def compute_metrics(y_true, y_scores, threshold=None):
    if threshold is None:
        threshold = np.percentile(y_scores[y_true == 0], 95)
    y_pred = (y_scores > threshold).astype(int)
    metrics = {
        "ACC": accuracy_score(y_true, y_pred),
        "PRE": precision_score(y_true, y_pred, zero_division=0),
        "REC": recall_score(y_true, y_pred, zero_division=0),
        "MCC": matthews_corrcoef(y_true, y_pred),
    }
    metrics["AUC"] = roc_auc_score(y_true, y_scores)
    return metrics, threshold

def find_best_f1_threshold(y_true, y_scores):
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_idx = np.argmax(f1_scores)
    return thresholds[best_idx], f1_scores[best_idx]

STATES_ORDER = [
    "Australian Capital Territory", "New South Wales", "Tasmania", "Western Australia"
]
s1_train = pd.read_csv(os.path.join(target_dir, "S1_scaled_train.csv"))
s1_train["State"]      = pd.Categorical(s1_train["State"], categories=STATES_ORDER, ordered=True)
s1_train["State_Code"] = s1_train["State"].cat.codes

logs_dir_s1 = os.path.join(target_dir, "00_1_training", "Model_A")

input_dim = len(NUMERIC_FEATURES)
embedding_dim = 4
hidden_dim = 32
num_states = s1_train[CAT_FEATURE].nunique()
layer_sizes = [256, 128, 64, 32, 16]
dropout = 0.1

s1_train["Year"] = pd.to_datetime(s1_train["Date"]).dt.year # type: ignore
years = list(range(2005, 2017))
for model_type in ["LSTM-AE", "Bi-LSTM-AE"]:
    for seq_name, seq_len in SEQUENCE_LENGTHS.items():
        print(f"Training {model_type} with sequence length: {seq_name}")
        for i in range(len(years) - 1):
            val_years   = [years[i], years[i + 1]]
            train_years = [y for y in years if y not in val_years]

            fold_train = s1_train[s1_train["Year"].isin(train_years)]
            fold_val   = s1_train[s1_train["Year"].isin(val_years)]
            
            train_ds = WildfireDataset(fold_train, seq_len, only_no_fire=True)
            val_ds = WildfireDataset(fold_val, seq_len, only_no_fire=False)

            train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, generator = g)
            val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

            model = LSTMAutoencoder(
                input_dim, embedding_dim, layer_sizes, num_states,
                bidirectional=(model_type == "Bi-LSTM-AE"), dropout=dropout
            )
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            criterion = nn.MSELoss()

            best_score = -float('inf')
            smoothed_best_score = -float('inf')
            min_epoch = 15
            patience, counter = 10, 0

            best_metrics = None
            best_threshold = None
            best_epoch = 0

            # Log path
            log_dir = os.path.join(logs_dir_s1, f"{model_type}")
            log_path = os.path.join(log_dir, f"01_train_{seq_name}_log.txt")

            smoothing_alpha = 0.1 # smoothing factor for exponential moving average
            smoothed_score = None

            with open(log_path, 'a') as f:
                log_and_print(f"\n{'='*50}", f)
                log_and_print(f"Training {model_type} with sequence length: {seq_name} | Fold {i+1} (years: {val_years})", f)
                for epoch in range(1, 201):
                    loss = train_one_epoch(model, train_loader, optimizer, criterion)
                    val_scores, val_labels = evaluate_model(model, val_loader)
                    threshold, _ = find_best_f1_threshold(val_labels, val_scores)
                    val_metrics, _ = compute_metrics(val_labels, val_scores, threshold)

                    val_loss = np.mean(val_scores)

                    mcc = val_metrics["MCC"]
                    rec = val_metrics["REC"]
                    pre = val_metrics["PRE"]
                    auc = val_metrics["AUC"]

                    scaled_mcc = (mcc + 1) / 2   # MCC: [-1, 1] → [0, 1]
                    scaled_auc = (auc - 0.5) * 2 # AUC: [0.5, 1] → [0, 1]

                    combo_score = 0.6 * scaled_mcc + 0.05 * rec + 0.05 * pre + 0.05 * scaled_auc
                    if smoothed_score is None:
                        smoothed_score = combo_score
                    else:
                        smoothed_score = (smoothing_alpha * combo_score + (1 - smoothing_alpha) * smoothed_score)

                    log_and_print(f"Epoch {epoch:02d} | Train loss: {loss:.5f} | Val loss: {val_loss:.5f} | "
                                f"Combo score: {combo_score:.3f} | Smoothed: {smoothed_score:.3f} | "
                                f"MCC: {mcc:.3f} | REC: {rec:.3f} | PRE: {pre:.3f} | AUC: {auc:.3f} | Threshold: {threshold:.5f}", f)

                    if combo_score > best_score or smoothed_score > smoothed_best_score:
                        smoothed_best_score = smoothed_score
                        counter = 0
                        # Always track best model based on raw combo score
                        if combo_score > best_score:
                            best_score = combo_score
                            best_metrics = val_metrics
                            best_threshold = threshold
                            best_epoch = epoch
                            torch.save(model.state_dict(), os.path.join(log_dir, f"01_train_{seq_name}_fold{i+1}_best_model.pt"))
                    # Early stopping kicks in only after min_epoch
                    elif epoch >= min_epoch:
                        counter += 1
                        if counter >= patience:
                            log_and_print(f"Early stopping at epoch {epoch:02d} (no improvement in smoothed or combo score over {patience} epochs)", f)
                            break

                # Summary logging
                log_and_print(f"\n--- Fold {i+1} summary (years: {val_years}) ---", f)
                print(f"Number of fire events in validation: ({(val_labels == 1).sum()} / {len(val_labels)})") # type: ignore
                log_and_print(f"Best epoch: {best_epoch}", f)
                if best_threshold is not None:
                    log_and_print(f"Threshold used: {best_threshold:.5f}", f)
                else:
                    log_and_print("Threshold used: N/A", f)
                if best_metrics is not None:
                    log_and_print("Validation metrics:", f)
                    for k, v in best_metrics.items():
                        if k in ["ACC", "PRE", "REC"]:
                            log_and_print(f"{k}: {v * 100:04.1f}%", f)
                        elif k in ["MCC", "AUC"]:
                            log_and_print(f"{k}: {v:.2f}", f)
                else:
                    log_and_print("No metrics saved.", f)

                log_and_print("=" * 50 + "\n", f)

print("Model A training and evaluation complete.")

Training LSTM-AE with sequence length: 10_day

Training LSTM-AE with sequence length: 10_day | Fold 1 (years: [2005, 2006])
Epoch 01 | Train loss: 0.05986 | Val loss: 0.04001 | Combo score: 0.396 | Smoothed: 0.396 | MCC: 0.183 | REC: 0.324 | PRE: 0.156 | AUC: 0.669 | Threshold: 0.09800
Epoch 02 | Train loss: 0.02144 | Val loss: 0.02132 | Combo score: 0.389 | Smoothed: 0.395 | MCC: 0.111 | REC: 0.755 | PRE: 0.057 | AUC: 0.649 | Threshold: 0.01994
Epoch 03 | Train loss: 0.01497 | Val loss: 0.01869 | Combo score: 0.413 | Smoothed: 0.397 | MCC: 0.181 | REC: 0.461 | PRE: 0.119 | AUC: 0.803 | Threshold: 0.03129
Epoch 04 | Train loss: 0.01244 | Val loss: 0.01714 | Combo score: 0.396 | Smoothed: 0.397 | MCC: 0.146 | REC: 0.471 | PRE: 0.093 | AUC: 0.736 | Threshold: 0.02647
Epoch 05 | Train loss: 0.01161 | Val loss: 0.01642 | Combo score: 0.402 | Smoothed: 0.397 | MCC: 0.158 | REC: 0.471 | PRE: 0.101 | AUC: 0.755 | Threshold: 0.02668
Epoch 06 | Train loss: 0.01104 | Val loss: 0.01684 | Combo sc

### **Training Models B (S2)**

In [ ]:
# MODEL B: Wildfire Prediction using LSTM-AE and Bi-LSTM-AE on Set 2 (S2)

random.seed(36)
np.random.seed(36)
torch.manual_seed(36)
torch.cuda.manual_seed_all(36)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False
g = torch.Generator().manual_seed(36)

SEQUENCE_LENGTHS = {
    "10_day": 10,
    "2_week": 14,
    "monthly": 30
}

NUMERIC_FEATURES = [
    "NDVI_residual", "NDVI_std", "NDVI_trend", "Solar_de-seasonalized",
    "Temperature_residual", "Humidity_trend", "NDVI_var", "Humidity_residual",
    "NDVI_mean", "Temperature_trend", "Temperature [°C]_max",
    "Solar radiation [Jm2/day]_std", "Relative humidity [%]_mean",
    "Temperature [°C]_mean", "Solar radiation [Jm2/day]_sum" #### these two were added later
]

CAT_FEATURE = "State"
TARGET_COL  = "fire"

class WildfireDataset(Dataset):
    def __init__(self, df, sequence_length, only_no_fire=True):
        self.sequence_length = sequence_length
        self.data = df.copy()
        self.features = NUMERIC_FEATURES
        self.cat_feature = CAT_FEATURE
        self.target = df[TARGET_COL].values
        grouped = [group for _, group in df.groupby('State', observed=False)]
        self.sequences = []
        self.targets = []
        for g in grouped:
            for i in range(len(g) - sequence_length):
                num_seq = g[self.features].iloc[i:i + sequence_length].values.astype(np.float32)
                cat_seq = g[self.cat_feature].iloc[i:i + sequence_length]
                cat_seq_encoded = cat_seq.astype('category').cat.codes.values.astype(np.float32).reshape(-1, 1)
                full_seq = np.hstack([num_seq, cat_seq_encoded])
                label = g[TARGET_COL].iloc[i + sequence_length - 1] # It predicts the last day of the sequence as fire/no-fire
                window_labels = g[TARGET_COL].iloc[i : i + sequence_length].values
                if only_no_fire and np.any(window_labels != 0):
                    # skip *any* sequence that contains a fire anywhere
                    continue
                self.sequences.append(full_seq)
                self.targets.append(label)
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.float32), torch.tensor(self.targets[idx], dtype=torch.float32)

class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, layer_sizes, num_states, bidirectional=False, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(num_states, embedding_dim)
        self.embedding_dropout = nn.Dropout(dropout)
        # Encoder
        enc_input_dim = input_dim + embedding_dim
        self.encoder_layers = nn.ModuleList()
        self.encoder_dropouts = nn.ModuleList()
        for h in layer_sizes:
            self.encoder_layers.append(
                nn.LSTM(enc_input_dim, h, batch_first=True, bidirectional=bidirectional)
            )
            self.encoder_dropouts.append(nn.Dropout(dropout))
            enc_input_dim = h * (2 if bidirectional else 1)
        # Decoder
        dec_input_dim = enc_input_dim
        self.decoder_layers = nn.ModuleList()
        self.decoder_dropouts = nn.ModuleList()
        for h in reversed(layer_sizes[:-1]):
            self.decoder_layers.append(
                nn.LSTM(dec_input_dim, h, batch_first=True)
            )
            self.decoder_dropouts.append(nn.Dropout(dropout))
            dec_input_dim = h
        # Final projection
        self.output_layer = nn.Linear(dec_input_dim, input_dim + embedding_dim)
        self.tanh = nn.Tanh()
    def forward(self, x):
        # Embedding
        state_idx = x[..., -1].long()
        emb = self.embedding(state_idx)
        emb = self.embedding_dropout(emb)
        seq = torch.cat([x[..., :-1], emb], dim=-1)
        # Encoder
        out = seq
        for lstm, dropout in zip(self.encoder_layers, self.encoder_dropouts):
            out, _ = lstm(out)
            out = dropout(self.tanh(out))
        # Decoder
        for lstm, dropout in zip(self.decoder_layers, self.decoder_dropouts):
            out, _ = lstm(out)
            out = dropout(self.tanh(out))
        # Output
        decoded = self.output_layer(out)
        decoded = self.tanh(decoded)
        return decoded

def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for x, _ in dataloader:
        optimizer.zero_grad()
        x_hat = model(x)
        # Trim output and target to match dimensions
        x_hat_trimmed = x_hat[:, :, :-embedding_dim]
        x_trimmed = x[:, :, :-1]
        loss = criterion(x_hat_trimmed, x_trimmed)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_scores = []
    with torch.no_grad():
        for x, y in dataloader:
            x_hat = model(x)
            # Trim output and target to match dimensions
            x_hat_trimmed = x_hat[:, :, :-embedding_dim]
            x_trimmed = x[:, :, :-1]
            re = ((x_hat_trimmed - x_trimmed) ** 2).mean(dim=(1, 2))
            all_scores.extend(re.numpy())
            all_labels.extend(y.numpy())
    return np.array(all_scores), np.array(all_labels)

def compute_metrics(y_true, y_scores, threshold=None):
    if threshold is None:
        threshold = np.percentile(y_scores[y_true == 0], 95)
    y_pred = (y_scores > threshold).astype(int)
    metrics = {
        "ACC": accuracy_score(y_true, y_pred),
        "PRE": precision_score(y_true, y_pred, zero_division=0),
        "REC": recall_score(y_true, y_pred, zero_division=0),
        "MCC": matthews_corrcoef(y_true, y_pred),
    }
    metrics["AUC"] = roc_auc_score(y_true, y_scores)
    return metrics, threshold

def find_best_f1_threshold(y_true, y_scores):
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_idx = np.argmax(f1_scores)
    return thresholds[best_idx], f1_scores[best_idx]

ALL_STATES_ORDER = [
    "Queensland", "South Australia", "Victoria", 
    "Australian Capital Territory", "New South Wales", "Tasmania", "Western Australia"
]
s2_train = pd.read_csv(os.path.join(target_dir, "S2_scaled_train.csv"))
s2_train["State"] = pd.Categorical(s2_train["State"], categories=ALL_STATES_ORDER, ordered=True)
s2_train["State_Code"] = s2_train["State"].cat.codes

logs_dir_s2 = os.path.join(target_dir, "00_1_training", "Model_B")

input_dim = len(NUMERIC_FEATURES)
embedding_dim = 4
hidden_dim = 32
num_states = s2_train[CAT_FEATURE].nunique()
layer_sizes = [256, 128, 64, 32, 16]
dropout = 0.1

s2_train["Year"] = pd.to_datetime(s2_train["Date"]).dt.year # type: ignore
years = list(range(2005, 2017))
for model_type in ["LSTM-AE", "Bi-LSTM-AE"]:
    for seq_name, seq_len in SEQUENCE_LENGTHS.items():
        print(f"Training {model_type} with sequence length: {seq_name}")
        for i in range(len(years) - 1):
            val_years   = [years[i], years[i + 1]]
            train_years = [y for y in years if y not in val_years]

            fold_train = s2_train[s2_train["Year"].isin(train_years)]
            fold_val   = s2_train[s2_train["Year"].isin(val_years)]
            
            train_ds = WildfireDataset(fold_train, seq_len, only_no_fire=True)
            val_ds = WildfireDataset(fold_val, seq_len, only_no_fire=False)

            train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, generator = g)
            val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

            model = LSTMAutoencoder(
                input_dim, embedding_dim, layer_sizes, num_states,
                bidirectional=(model_type == "Bi-LSTM-AE"), dropout=dropout
            )
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            criterion = nn.MSELoss()

            best_score = -float('inf')
            smoothed_best_score = -float('inf')
            min_epoch = 15
            patience, counter = 10, 0

            best_metrics = None
            best_threshold = None
            best_epoch = 0

            # Log path
            log_dir = os.path.join(logs_dir_s1, f"{model_type}")
            log_path = os.path.join(log_dir, f"01_train_{seq_name}_log.txt")

            smoothing_alpha = 0.1 # smoothing factor for exponential moving average
            smoothed_score = None

            with open(log_path, 'a') as f:
                log_and_print(f"\n{'='*50}", f)
                log_and_print(f"Training {model_type} with sequence length: {seq_name} | Fold {i+1} (years: {val_years})", f)

                for epoch in range(1, 201):
                    loss = train_one_epoch(model, train_loader, optimizer, criterion)
                    val_scores, val_labels = evaluate_model(model, val_loader)
                    threshold, _ = find_best_f1_threshold(val_labels, val_scores)
                    val_metrics, _ = compute_metrics(val_labels, val_scores, threshold)

                    val_loss = np.mean(val_scores)

                    mcc = val_metrics["MCC"]
                    rec = val_metrics["REC"]
                    pre = val_metrics["PRE"]
                    auc = val_metrics["AUC"]

                    scaled_mcc = (mcc + 1) / 2   # MCC: [-1, 1] → [0, 1]
                    scaled_auc = (auc - 0.5) * 2 # AUC: [0.5, 1] → [0, 1]

                    combo_score = 0.6 * scaled_mcc + 0.05 * rec + 0.05 * pre + 0.05 * scaled_auc
                    if smoothed_score is None:
                        smoothed_score = combo_score
                    else:
                        smoothed_score = (smoothing_alpha * combo_score + (1 - smoothing_alpha) * smoothed_score)

                    log_and_print(f"Epoch {epoch:02d} | Train loss: {loss:.5f} | Val loss: {val_loss:.5f} | "
                                f"Combo score: {combo_score:.3f} | Smoothed: {smoothed_score:.3f} | "
                                f"MCC: {mcc:.3f} | REC: {rec:.3f} | PRE: {pre:.3f} | AUC: {auc:.3f} | Threshold: {threshold:.5f}", f)

                    if combo_score > best_score or smoothed_score > smoothed_best_score:
                        smoothed_best_score = smoothed_score
                        counter = 0
                        # Always track best model based on raw combo score
                        if combo_score > best_score:
                            best_score = combo_score
                            best_metrics = val_metrics
                            best_threshold = threshold
                            best_epoch = epoch
                            torch.save(model.state_dict(), os.path.join(log_dir, f"01_train_{seq_name}_fold{i+1}_best_model.pt"))
                    # Early stopping kicks in only after min_epoch
                    elif epoch >= min_epoch:
                        counter += 1
                        if counter >= patience:
                            log_and_print(f"Early stopping at epoch {epoch:02d} (no improvement in smoothed or combo score over {patience} epochs)", f)
                            break

                # Summary logging
                log_and_print(f"\n--- Fold {i+1} summary (years: {val_years}) ---", f)
                print(f"Number of fire events in validation: ({(val_labels == 1).sum()} / {len(val_labels)})")
                log_and_print(f"Best epoch: {best_epoch}", f)
                if best_threshold is not None:
                    log_and_print(f"Threshold used: {best_threshold:.5f}", f)
                else:
                    log_and_print("Threshold used: N/A", f)
                if best_metrics is not None:
                    log_and_print("Validation metrics:", f)
                    for k, v in best_metrics.items():
                        if k in ["ACC", "PRE", "REC"]:
                            log_and_print(f"{k}: {v * 100:04.1f}%", f)
                        elif k in ["MCC", "AUC"]:
                            log_and_print(f"{k}: {v:.2f}", f)
                else:
                    log_and_print("No metrics saved.", f)

                log_and_print("=" * 50 + "\n", f)

print("Model B training and evaluation complete.")

Training LSTM-AE with sequence length: 10_day

Training LSTM-AE with sequence length: 10_day | Fold 1 (years: [2005, 2006])
Epoch 01 | Train loss: 0.04699 | Val loss: 0.02192 | Combo score: 0.403 | Smoothed: 0.403 | MCC: 0.159 | REC: 0.653 | PRE: 0.153 | AUC: 0.653 | Threshold: 0.02221
Epoch 02 | Train loss: 0.01538 | Val loss: 0.01906 | Combo score: 0.399 | Smoothed: 0.403 | MCC: 0.165 | REC: 0.453 | PRE: 0.185 | AUC: 0.676 | Threshold: 0.02623
Epoch 03 | Train loss: 0.01295 | Val loss: 0.01664 | Combo score: 0.413 | Smoothed: 0.404 | MCC: 0.194 | REC: 0.559 | PRE: 0.187 | AUC: 0.678 | Threshold: 0.01989
Epoch 04 | Train loss: 0.01086 | Val loss: 0.01606 | Combo score: 0.415 | Smoothed: 0.405 | MCC: 0.211 | REC: 0.461 | PRE: 0.221 | AUC: 0.678 | Threshold: 0.02464
Epoch 05 | Train loss: 0.01020 | Val loss: 0.01503 | Combo score: 0.403 | Smoothed: 0.405 | MCC: 0.184 | REC: 0.382 | PRE: 0.218 | AUC: 0.680 | Threshold: 0.02498
Epoch 06 | Train loss: 0.00981 | Val loss: 0.01457 | Combo sc